# Final Project - Programming For Data Science - 21KHDL1

## VNU HCM - University of Science
---

## Member Information
| Name              | ID       |
|-------------------|----------|
| Tran Nguyen Huan  | 21127050 |
| Nguyen Phat Dat   | 21127240 |

# Table of Contents

## Overview
---

<h3>
    <b>
    US Accidents (2016 - 2023)
    </b>
</h3>
    <img style="padding:10px" src="https://hire.refactored.ai/upload-nct/portfolio_images/253/1626148168_GqMh6a2U.png" width="800"/>
</center>
This is a countrywide car accident dataset that were collected using multiple APIs that provide streaming traffic incident (or event) data. These APIs broadcast traffic data captured by various entities, including the US and state departments of transportation, law enforcement agencies, traffic cameras, and traffic sensors within the road networks.

The data covers _49 states of the USA_ from _February 2016 to March 2023_. 

Note that the original dataset contains approximately __7.7 million records__ which is too large for our purpose (mostly data analysis). Therefore, we will just use a sampled version of this dataset which contains only __500.000 accident records__.

Visit [here](https://smoosavi.org/datasets/us_accidents) for more information about the dataset.

This Dataset can be accessed [here](https://doi.org/10.34740/KAGGLE/DS/199387): Sobhan Moosavi. (2023). <i>US Accidents (2016 - 2023)</i> Kaggle. Or [here](https://drive.google.com/file/d/1U3u8QYzLjnEaSurtZfSAS_oh9AT2Mn8X) for the sampled data.
<center>


__Goal:__ The primary goal of the project is to analyze and generate insights on the traffic accidents that took place in USA from Feb. 2016 to Mar. 2023.

## Usage Policy and Legal Disclaimer
---
This dataset is being distributed solely for research purposes under the Creative Commons Attribution-Noncommercial-ShareAlike license (CC BY-NC-SA 4.0). By downloading the dataset, we agree to use it only for non-commercial, research, or academic applications.

## Code Environment
---

In [7]:
import sys
sys.executable

'/home/datlinux/anaconda3/envs/min_ds-env/bin/python'

## Import necessary libraries
---

In [2]:
import numpy as np
import pandas as pd
from pprint import pprint

## Dataset Import
---

In the first place we are going to import the dataset using Pandas module.

In [3]:
df = pd.read_csv("Data/US_Accidents_March23_sampled_500k.csv")

Let's take a quick look at our data by examining the number of rows, columns.  

In [4]:
# print out the shape to know number of rows and columns
print("Shape:", df.shape)

Shape: (500000, 46)


There are __500.000__ records of accidents in our data with each has __46__ features about that accident.

The number of features here are a little too large, which might result in truncating when the data is being displayed. Therefore, we will set a larger the number of max columns in case this is not handled automatically.

In [5]:
# Set the Pandas display options to show all columns
pd.set_option('display.max_columns', 50)

Then, print out some samples to get a quick insight from our data.

In [6]:
# examine 4 random samples from the data
df.sample(4)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
37356,A-7237468,Source1,2,2020-01-24 13:44:00,2020-01-24 14:53:15,34.121559,-117.696732,34.121559,-117.696732,0.000,At Towne Ave/Exit 50 - Accident.,CA-210 W,Claremont,Los Angeles,CA,91711,US,US/Pacific,KPOC,2020-01-24 13:47:00,72.0,72.0,35.0,28.90,10.0,W,7.0,NaN,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
338775,A-5349827,Source1,2,2023-01-25 08:08:00,2023-01-25 10:36:30,45.040444,-93.096902,45.066832,-93.240968,7.264,Slow traffic on I-694 W - US-10 W from I-35E/I...,I-694 W,Saint Paul,Ramsey,MN,55127,US,US/Central,KANE,2023-01-25 07:45:00,27.0,21.0,100.0,28.87,5.0,NNW,5.0,0.0,Light Snow,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
399121,A-1557969,Source2,2,2020-04-17 08:34:12,2020-04-17 09:52:16,35.869701,-78.497253,NaN,NaN,0.000,Accident on Watkins Rd at Jonesbay Ln.,Watkins Town Rd,Raleigh,Wake,NC,27616-8585,US,US/Eastern,KRDU,2020-04-17 08:51:00,53.0,53.0,59.0,29.88,10.0,CALM,0.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
274196,A-5618148,Source1,2,2021-04-02 04:44:30,2021-04-02 17:54:40,37.664411,-97.482077,37.665613,-97.480609,0.116,Slow traffic on US-54 E - US-400 E from S Horn...,W Harry St,Wichita,Sedgwick,KS,67235,US,US/Central,KICT,2021-04-02 04:53:00,39.0,31.0,55.0,28.85,10.0,SSE,13.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night


## Features explanation
There are __46__ features in our dataset, include:
1. `ID`: A unique identifier of the accident record.
2. `Source`: Source of raw accident data (it were collected from two different sources).
3. `Severity`: The severity of the accident by a number between 1 and 4, where 1 indicates the least impact on traffic (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay).
4. `Start_Time`: Start time of the accident in local time zone.
5. `End_Time`: Refers to when the impact of accident on traffic flow was dismissed.
6. `Start_Lat`: Latitude in GPS coordinate of the start point.
7. `Start_Lng`: Longitude in GPS coordinate of the start point.
8. `End_Lat`: same as above but for the end point.
9. `End_Lng`: same as above but for the end point.
10. `Distance(mi)`: The length of the road extent affected by the accident (in _miles_).
11. `Description`: Shows a human provided description of the accident.
12. `Street`: Shows the street name in address field.
13. `City`: City name.
14. `County`: Country name.
15. `State`: State name.
16. `Zipcode`: Zipcode.
17. `Country`: Country in address field (this field is different from above field).
18. `Timezone`: Shows timezone based on the location of the accident (eastern, central, etc.).
19. `Airport_Code`: Denotes an airport-based weather station which is the closest one to location of the accident.
20. `Weather_Timestamp`: Time-stamp of weather observation record.
21. `Temperature(F)`: Temperature (in Fahrenheit).
22. `Wind_Chill(F)`: Wind chill (in Fahrenheit).
23. `Humidity(%)`: Humidity (in percentage).
24. `Pressure(in)`: Air pressure (in inches).
25. `Visibility(mi)`: Visibility (in miles).
26. `Wind_Direction`: Wind direction.
27. `Wind_Speed(mph)`: Wind speed (in miles per hour).
28. `Precipitation(in)`: Precipitation amount in inches.
29. `Weather_Condition`: Weather condition (rain, snow, thunderstorm, fog, etc.)
30. `Amenity`: Amenity in a nearby location (true/false).
31. `Bump`: Speed bump or hump in a nearby location (true/false).
32. `Crossing`: Crossing in a nearby location (true/false).
33. `Give_Way`: Give_way in a nearby location (true/false).
34. `Junction`: Junction in a nearby location (true/false).
35. `No_Exit`: No_exit in a nearby location (true/false).
36. `Railway`: Railway in a nearby location (true/false).
37. `Roundabout`: Roundabout in a nearby location (true/false).
38. `Station`: Station in a nearby location (true/false).
39. `Stop`: Stop in a nearby location (true/false).
40. `Traffic_Calming`: Traffic_calming in a nearby location (true/false).
41. `Traffic_Signal`: Traffic_signal in a nearby location (true/false).
42. `Turning_Loop`: Turning_loop in a nearby location (true/false).
43. `Sunrise_Sunset`: based on sunrise/sunset (day/night).
44. `Civil_Twilight`: based on civil twilight (day/night).
45. `Nautical_Twilight`: based on nautical twilight (day/night).
46. `Astronomical_Twilight`: based on astronomical twilight (day/night).

## Data Cleansing
---

This dataset contains a large amount of information for analysis. However, some of the fields may be overly complex and not contribute significantly to our analysis. Before proceeding further, I plan to streamline the dataset by removing the following fields:

1. `ID`: We will replace the this field with the simple numeric index to reduce memory consumption.

2. `Source`: This field do not provide substantial information for our analysis.

3. `End_Lat` and `End_Lng`: Because normally we only need to know where the accidents happen, so the starting coordinates (`Start_Lat` and `Start_Lng`) are enough. However, we can consider using those ending points of the accidents in some other problems.

4. `Airport_Code`: Specifying the nearest airport code is unnecessary in our problem.

5. `Country`: As mentioned earlier, all the data is related to the USA, so this field does not add value.

6. `Weather_Timestamp`: This field is not neccessary since we have `Start_Time` and `End_Time`.

7. `Civil_Twilight`, `Nautical_Twilight`, and `Astronomical_Twilight`: These fields may not be directly relevant to our analysis, since we can derive the period of day (_Day_ and _Night_ ) directly from the time data (`Start_Time` and `End_Time`) and our analysis mostly focus on normal people.

8. `Timezone`: This information can be derived from other relevant fields.

9. `Description`: This field may be neccessary in some cases (to get a quick insight about the accident), but the field is provided by human natural language which is not easy to be processed and this task is also out of our scope in this project.

By removing these fields, we aim to simplify the dataset, making it more focused and efficient for our analysis.

In [ ]:
# Specify the names of the columns to be dropped
cols2drop = ['End_Lat', 'End_Lng', 'ID', 'Source', 'Airport_Code', 'Country', 'Weather_Timestamp', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Timezone']

# Use the drop() method to remove the specified columns
df.drop(columns=cols2drop, inplace=True)

**Quick overview of the Data:**

In the following, a function is defined to show detailed overview of our data, and this function can be reused. 

In [ ]:
from pprint import pprint

def sanity_check(df):
    pprint('-' * 70)
    pprint('No. of Rows: {0[0]}        No. of Columns : {0[1]}'.format(df.shape))
    pprint('-' * 70)
    data_profile = pd.DataFrame(df.dtypes.reset_index()).rename(columns={'index': 'Attribute', 0: 'DataType'}).set_index('Attribute')

    data_profile = pd.concat([data_profile, df.isnull().sum()], axis=1).rename(columns={0: 'Missing Values'})
    data_profile = pd.concat([data_profile, (df.isnull().mean() * 100).round(2)], axis=1).rename(columns={0: 'Missing %'})
    data_profile = pd.concat([data_profile, df.nunique()], axis=1).rename(columns={0: 'Unique Values'})

    # Additional features for numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    numeric_stats = df[numeric_cols].describe().transpose()[['min', '25%', '50%', '75%', 'max']]
    data_profile = pd.concat([data_profile, numeric_stats], axis=1).rename(columns={'min': 'Min', '25%': 'Q1', '50%': 'Median', '75%': 'Q3', 'max': 'Max'})

    pprint(data_profile)
    pprint('-' * 70)

# Example usage:
sanity_check(df)


For the columns, **Precipitation(in), Wind_Chill(F), and Wind_Speed(mph)**, the missing data is in high percentage, removing missing data from these columns would cause us to lose a lot of data (around 3 million records). Therefore, we are going to impute them with the mean values of those fields.

In [ ]:
df.dropna(subset=['Visibility(mi)', 'Wind_Direction', 'Description', 'Humidity(%)', 'Weather_Condition', 'Temperature(F)', 'Pressure(in)', 'Sunrise_Sunset', 'Street', 'Zipcode'], inplace=True)

For the columns, **Precipitation(in), Wind_Chill(F), and Wind_Speed(mph)**, the missing data is in high percentage, removing missing data from these columns would cause us to lose a lot of data (around 3 million records). Therefore, we are going to impute them with the mean values of those fields.

In [ ]:
columns = ['Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)']

for c in columns:
    df[c].fillna(df[c].mean(), inplace=True)

Let's run the sanity check on the modified data.

In [ ]:
sanity_check(df)

It's time to remove duplicate rows.

In [ ]:
print("Number of rows:", len(df.index))
df.drop_duplicates(inplace=True)
print("Number of rows after dropping duplicates:", len(df.index))

## Exploring Accidents: A Deep Dive into Data Insights (Raise questions that need answering)
---

### Question 1:

#### The purpose of the question

#### How to solve the question

#### Pre-processing

#### Exploratory Analysis and Visualization

#### Answer the question

### Question 2:

#### The purpose of the question

#### How to solve the question

#### Pre-processing

#### Exploratory Analysis and Visualization

#### Answer the question

### Question 3:

#### The purpose of the question

#### How to solve the question

#### Pre-processing

#### Exploratory Analysis and Visualization

#### Answer the question

### Question 4:

#### The purpose of the question

#### How to solve the question

#### Pre-processing

#### Exploratory Analysis and Visualization

#### Answer the question

## Reflection

## References

* https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents

* Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. [“A Countrywide Traffic Accident Dataset.”](https://arxiv.org/abs/1906.05409), 2019.

* Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. ["Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights."](https://arxiv.org/abs/1909.09638) In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.